In [200]:
import psycopg2
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from sqlalchemy import create_engine, text
from tqdm import tqdm
from geopy.distance import geodesic

In [4]:
# Define database connection parameters
database_connection = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'vic_db',
}


# A psycopg2 connection and cursor
conn = psycopg2.connect(user=database_connection['username'],
                        password=database_connection['password'],
                        host=database_connection['host'],
                        port=database_connection['port'],
                        database=database_connection['database'])
conn.autocommit = True
cursor = conn.cursor()

# Create a SQLAlchemy engine
engine = create_engine('postgresql://%(username)s:%(password)s@%(host)s/%(database)s' % database_connection, isolation_level="AUTOCOMMIT")
conn_alchemy = engine.connect()

In [41]:

# Define the SQL query to select data from the PostGIS table
sql = "SELECT * FROM vmtrans.tr_road_all LIMIT 10;"

# Read data from PostGIS into a GeoDataFrame
gdf = gpd.read_postgis(sql, con=engine)

# # Print the GeoDataFrame
# print(gdf)


In [9]:
# List all tables
cursor.execute("SELECT table_schema, table_name FROM information_schema.tables;")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [11]:
cursor.execute("SELECT * FROM vmtrans.tr_road_all LIMIT 10;")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [37]:
# In table tr_road_all, create a column called "road_length" which is the length of the road segment
# But not really create it, just create a new column from SELECT statement. Use meters as unit, so we have to convert the length from degree to meters
# cursor.execute("SELECT SUM(ST_Length(geom::geography)) as road_length FROM vmtrans.tr_road_all WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';")
cursor.execute(
"""
SELECT 
    ST_Length(geom::geography) as road_length,
    ST_AsText(ST_StartPoint(ST_GeometryN(geom, 1))) AS first_point,
    ST_AsText(ST_EndPoint(ST_GeometryN(geom, 1))) AS last_point
FROM vmtrans.tr_road_all 
WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';
""")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
result_df

,road_length,first_point,last_point
0,74.321053,POINT(144.7845466330001 -37.765636496999946),POINT(144.78466143100002 -37.76497362999993)
1,114.023511,POINT(145.0549611360001 -37.86946304899993),POINT(145.05515306400002 -37.86844708399997)
2,271.300323,POINT(145.12387210600002 -37.90911573999995),POINT(145.12431451100008 -37.90669674099996)
3,52.212317,POINT(145.0548732430001 -37.86992826699998),POINT(145.0549611360001 -37.86946304899993)
4,247.812288,POINT(144.78573093800003 -37.76742263799997),POINT(144.7845466330001 -37.765636496999946)
5,13.516021,POINT(145.12301246200002 -37.91167085699993),POINT(145.12310854800012 -37.911575813999946)
6,287.039748,POINT(145.12310854800012 -37.911575813999946),POINT(145.12387210600002 -37.90911573999995)
7,17.342869,POINT(145.1290169560001 -38.05468524999998),POINT(145.12883521900005 -38.05474658899993)
8,301.855147,POINT(145.05515306400002 -37.86844708399997),POINT(145.0556611290001 -37.86575751099997)
9,164.729226,POINT(144.7879104740001 -37.76769546799994),POINT(144.7860614210001 -37.76747599199996)


In [59]:
# sql = """
# SELECT ezi_road_name, ST_Length(geom::geography) as road_length, ST_AsText(ST_StartPoint(ST_GeometryN(geom, 1))) AS first_point, ST_AsText(ST_EndPoint(ST_GeometryN(geom, 1))) AS last_point FROM vmtrans.tr_road_all WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';
# """
sql = """
SELECT
    GeometryType(geom) AS geometry_type,
    COUNT(*) AS count
FROM vmtrans.tr_road_all
GROUP BY GeometryType(geom);
"""

pd.read_sql_query(text(sql), conn_alchemy)

,geometry_type,count
0,MULTILINESTRING,1222415


In [68]:
sql = """
SELECT
    from_ufi,
    to_ufi,
    ST_X(first_point::geometry) AS first_point_x,
    ST_Y(first_point::geometry) AS first_point_y,
    ST_X(last_point::geometry) AS last_point_x,
    ST_Y(last_point::geometry) AS last_point_y
FROM vmtrans.tr_road_all
LIMIT 10;
"""
# gpd.read_postgis(text(sql), con=engine)
pd.read_sql_query(text(sql), conn_alchemy)

,from_ufi,to_ufi,first_point_x,first_point_y,last_point_x,last_point_y
0,2036059.0,2036057.0,144.632226,-36.182305,144.635463,-36.182305
1,15554142.0,16288188.0,144.982563,-37.834037,144.983084,-37.833828
2,18459581.0,18459580.0,145.239520,-38.448936,145.239522,-38.448835
3,16725603.0,16384630.0,144.332436,-37.398808,144.333167,-37.399693
4,2408990.0,16729012.0,144.352495,-38.134773,144.353832,-38.134926
5,16288145.0,16288142.0,144.974104,-37.828773,144.973387,-37.827878
6,13177738.0,2273314.0,146.056055,-37.726144,146.055975,-37.726238
7,15347009.0,15347010.0,144.729790,-36.133315,144.730937,-36.133320
8,13157990.0,13157986.0,144.340383,-36.800249,144.340994,-36.799485
9,16678757.0,16678759.0,145.150687,-37.339803,145.149248,-37.340359


In [ ]:
# Add new columns to the table
sql = """
ALTER TABLE vmtrans.tr_road_all
ADD COLUMN road_length_meters double precision,
ADD COLUMN from_point geometry(Point, 7844),
ADD COLUMN to_point geometry(Point, 7844);

UPDATE vmtrans.tr_road_all
SET road_length_meters = ST_Length(geom::geography),
    from_point = ST_StartPoint(ST_GeometryN(geom, 1)),
    to_point = ST_EndPoint(ST_GeometryN(geom, 1));
"""
cursor.execute(sql)
# 1m 30s

In [97]:
# vmtrans.tr_road_all has columns from_ufi, to_ufi, from_point, to_point
# Create a new table from vmtrans.tr_road_all, with 2 columns: ufi, point
# The new table should have 2 rows for each row in vmtrans.tr_road_all, one for from_ufi and from_point, one for to_ufi and to_point
sql = """
CREATE TABLE vmtrans.tr_points AS
SELECT from_ufi AS ufi, from_point AS geom FROM vmtrans.tr_road_all
UNION ALL
SELECT to_ufi AS ufi, to_point AS geom FROM vmtrans.tr_road_all;
"""
cursor.execute(sql)
# 17s - 30s

# Check that each ufi has only one point
sql = """
SELECT ufi, COUNT(DISTINCT geom) AS count
FROM vmtrans.tr_points
GROUP BY ufi
HAVING COUNT(DISTINCT geom) > 1;
"""
cursor.execute(sql)
result = cursor.fetchall()
assert len(result) == 0

# Remove duplicate ufi-geom pairs
sql = """
CREATE TABLE vmtrans.tr_points_clean AS
SELECT ufi, geom
FROM vmtrans.tr_points
GROUP BY ufi, geom;
"""
cursor.execute(sql)

# Rename the table from tr_points_clean to tr_points
sql = """
DROP TABLE IF EXISTS vmtrans.tr_points;
ALTER TABLE vmtrans.tr_points_clean
RENAME TO tr_points;
"""
cursor.execute(sql)

# Assert that ufi is unique
cursor.execute("SELECT COUNT(ufi), COUNT(DISTINCT ufi) FROM vmtrans.tr_points;")
result = cursor.fetchall()
assert result[0][0] == result[0][1]

In [98]:
gpd.read_postgis("SELECT * FROM vmtrans.tr_points LIMIT 10;", con=engine)

,ufi,geom
0,54637436.0,POINT (148.21369 -35.75781)
1,59918101.0,POINT (145.65433 -36.30990)
2,60425899.0,POINT (149.58132 -36.06935)
3,49754475.0,POINT (147.17442 -35.65763)
4,51705580.0,POINT (143.86532 -37.58262)
5,46722523.0,POINT (146.72891 -36.34118)
6,60416152.0,POINT (148.56595 -35.09645)
7,54757879.0,POINT (144.60838 -36.24235)
8,49692680.0,POINT (144.42042 -34.85279)
9,55247502.0,POINT (146.99632 -36.24295)


In [134]:
# Check that all ezi_road_name are just ezi_road_name_label in uppercase, with ' - ' replaced by '-'
sql = """
SELECT ezi_road_name, ezi_road_name_label
FROM vmtrans.tr_road_all
WHERE REPLACE(UPPER(ezi_road_name_label), ' - ', '-') != ezi_road_name;
"""
cursor.execute(sql)
result = cursor.fetchall()
assert len(result) == 0

In [ ]:
df = pd.read_sql_query(text("SELECT ufi, ezi_road_name, ezi_road_name_label, feature_type_code, direction_code, from_ufi, to_ufi, road_length_meters FROM vmtrans.tr_road_all WHERE feature_type_code = 'ferry_route' AND direction_code IS NOT NULL;"), conn_alchemy)
# B F R

In [146]:
# Implement A* algorithm to find the shortest path between two points
# The graph is represented by the table tr_points
# The cost of each edge is the road_length_meters
# The heuristic is the euclidean distance between two points
# The algorithm should return the path and the total cost

# Define the A* algorithm
frontier = []
visited = set()
path = []

def astar(start, goal):
    frontier.append((0, start, []))
    while frontier:
        cost, current, path = frontier.pop(0)
        if current == goal:
            return path, cost
        if current in visited:
            continue
        visited.add(current)
        for neighbor_point, road_ufi, road_length in neighbors(current):
            # frontier.append((cost + edge_cost(current, neighbor), neighbor, path + [neighbor]))
            if neighbor_point not in visited:
                frontier.append((cost + road_length, neighbor_point, path + [road_ufi]))
        frontier.sort(key=lambda x: x[0] + heuristic(x[1], goal))

# def neighbors(current):
#     sql = f"""
#     SELECT to_ufi, ufi, road_length_meters
#     FROM vmtrans.tr_road_all
#     WHERE from_ufi = {current}
#     AND (direction_code = 'B' OR direction_code = 'F')
#     """
#     cursor.execute(sql)
#     neighbors1 = cursor.fetchall()
#     sql = f"""
#     SELECT from_ufi, ufi, road_length_meters
#     FROM vmtrans.tr_road_all
#     WHERE to_ufi = {current}
#     AND (direction_code = 'B' OR direction_code = 'R')
#     """
#     cursor.execute(sql)
#     neighbors2 = cursor.fetchall()
#     return [(neighbor[0], neighbor[1], neighbor[2]) for neighbor in neighbors1 + neighbors2]


# def heuristic(current, goal):
#     sql = f"""
#     SELECT ST_Distance(
#         (SELECT geom FROM vmtrans.tr_points
#         WHERE ufi = {current}),
#         (SELECT geom FROM vmtrans.tr_points
#         WHERE ufi = {goal})
#     );
#     """
#     cursor.execute(sql)
#     return cursor.fetchall()[0][0]



In [148]:
points_coords = pd.read_sql_query(text("SELECT ufi, ST_X(geom::geometry) AS x, ST_Y(geom::geometry) AS y FROM vmtrans.tr_points;"), conn_alchemy)
points_coords.set_index('ufi', inplace=True)
points_coords = points_coords.to_dict(orient='index')
points_coords = {ufi: (coords['x'], coords['y']) for ufi, coords in points_coords.items()}
# 10s

In [160]:
roads = pd.read_sql_query(text("SELECT ufi, direction_code, from_ufi, to_ufi, road_length_meters FROM vmtrans.tr_road_all;"), conn_alchemy)

In [162]:
neighbors1 = roads[roads['direction_code'] == 'B'].groupby('from_ufi')[['ufi', 'to_ufi', 'road_length_meters']].apply(lambda x: x.to_dict(orient='records')).to_dict()

In [172]:
# Convert roads into a list of tuples
roads = roads.to_dict(orient='records')

In [178]:
neighbors = {}
with tqdm(total=len(roads)) as pbar:
    for i, road in enumerate(roads):
        if road['direction_code'] == 'B' or road['direction_code'] == 'F':
            if road['from_ufi'] not in neighbors:
                neighbors[road['from_ufi']] = []
            neighbors[road['from_ufi']].append((road['to_ufi'], road['ufi'], road['road_length_meters']))
        if road['direction_code'] == 'B' or road['direction_code'] == 'R':
            if road['to_ufi'] not in neighbors:
                neighbors[road['to_ufi']] = []
            neighbors[road['to_ufi']].append((road['from_ufi'], road['ufi'], road['road_length_meters']))
        pbar.update(1)

100%|██████████| 1222415/1222415 [00:12<00:00, 94370.80it/s] 


In [198]:
frontier = []
visited = set()
path = []

import shapely.geometry as sg

def heuristic(current, goal):
    # Get distance in real world from coordinates
    # Convert to meters
    # Calculate geodesic distance between two points in meters
    lon1, lat1 = points_coords[current]
    lon2, lat2 = points_coords[goal]
    distance = geodesic((lat1, lon1), (lat2, lon2)).meters
    return distance

def astar(start, goal):
    frontier.append((0, start, []))
    while frontier:
        cost, current, path = frontier.pop(0)
        print(cost, current)
        if current == goal:
            return path, cost
        if current in visited:
            continue
        visited.add(current)
        for neighbor_point, road_ufi, road_length in neighbors.get(current, []):
            # frontier.append((cost + edge_cost(current, neighbor), neighbor, path + [neighbor]))
            if neighbor_point not in visited:
                frontier.append((cost + road_length, neighbor_point, path + [road_ufi]))
        frontier.sort(key=lambda x: x[0] + heuristic(x[1], goal))

In [202]:
# Pick a random start and goal
start = np.random.choice(list(points_coords.keys()))
goal = np.random.choice(list(points_coords.keys()))
start, goal, heuristic(start, goal), points_coords[start], points_coords[goal]

(36734996.0,
 60284854.0,
 184276.86740350578,
 (145.24577906600007, -38.47207933999994),
 (143.15736838300006, -38.24407185699994))

In [203]:
path, cost = astar(start, goal)

0 36734996.0
94.12526654458813 36734989.0
138.42352666752691 36734990.0
153.0934756077563 36734991.0
216.17593979961435 36734992.0
254.43254582717896 36734987.0
341.83571750699855 36734986.0
183.40257922731072 36734991.0
359.562365507278 36734988.0
415.55200601631907 36734993.0
433.2134559633953 36734994.0
282.78386093715477 36734987.0
272.93943916580514 36734999.0
514.0016099513194 36735000.0
450.60536006012353 36734993.0
482.1220426437376 36734994.0
554.64676880113 36735003.0
690.9129363339932 36055491.0
319.46075748536987 36735005.0
79.38853453615053 36735002.0
606.9123402058033 36735003.0
204.97530455262566 36734985.0
114.20632025317732 36734998.0
140.24508297283373 36734997.0
390.1610573242576 36735015.0
438.09488778836965 36735024.0
427.49197231770404 36735021.0
481.5146581886246 36735027.0
185.3410304091276 36735007.0
217.79632138020128 36734995.0
209.53222367364384 36735010.0
225.772770798494 36735014.0
481.5398050635453 36735023.0
237.77419799737754 36735017.0
252.738135291955

In [144]:
# Find the shortest path between two points
# Take 2 random points from tr_points
sql = """
SELECT ufi
FROM vmtrans.tr_points
ORDER BY random()
LIMIT 2;
"""
cursor.execute(sql)
start, goal = cursor.fetchall()
start, goal = start[0], goal[0]

# path, cost = astar(start, goal)
# path, cost
start, goal

(54162190.0, 60409536.0)